# Демонстрация использования класса `SignalChecker`

Этот ноутбук показывает, как использовать класс `SignalChecker` из `analysis.signal_checker` для проверки наличия определенных наборов сигналов в файлах COMTRADE (`.cfg`).

Демонстрируются:
- Использование стандартной логики проверки сигналов (по умолчанию в классе).
- Использование пользовательской функции для проверки специфического набора сигналов.

**Важно**: Этот ноутбук использует образцы файлов из директории `tests/sample_data/`. Для демонстрации файлы копируются во временную рабочую директорию (`temp_signal_checker_notebook_data/...`). Убедитесь, что тестовые данные были сгенерированы запуском скрипта `tests/test_data_setup.py`.

In [ ]:
import os
import shutil
import pandas as pd
import sys
# import numpy as np # Not strictly needed for this notebook's setup if copying files

# --- Пути к данным для тестирования ---
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)
    
from analysis.signal_checker import SignalChecker

base_sample_data_dir = os.path.join(module_path, "tests", "sample_data")
comtrade_samples_path = os.path.join(base_sample_data_dir, "comtrade_files")

# --- Создание временной рабочей директории для этого ноутбука ---
notebook_base_temp_dir_sc = "temp_signal_checker_notebook_data"
notebook_source_dir_sc = os.path.join(notebook_base_temp_dir_sc, "source_oscillograms_sc")
# Директория для CSV отчетов
notebook_output_dir_sc = os.path.join(notebook_base_temp_dir_sc, "output_reports_sc") 

if os.path.exists(notebook_base_temp_dir_sc):
    shutil.rmtree(notebook_base_temp_dir_sc)
os.makedirs(notebook_source_dir_sc)
os.makedirs(notebook_output_dir_sc)

# --- Копирование необходимых файлов для демонстрации ---
# Файлы из tests/sample_data/comtrade_files/
    
# 1. Файл, который должен пройти стандартную проверку 
#    (signals_for_checker.cfg из specific_signals_1/)
src_valid_cfg_path = os.path.join(comtrade_samples_path, "specific_signals_1", "signals_for_checker.cfg")
src_valid_dat_path = os.path.join(comtrade_samples_path, "specific_signals_1", "signals_for_checker.dat")
dest_valid_cfg_name = "osc_sc_valid_from_testdata.cfg" 

if os.path.exists(src_valid_cfg_path) and os.path.exists(src_valid_dat_path):
    shutil.copy(src_valid_cfg_path, os.path.join(notebook_source_dir_sc, dest_valid_cfg_name))
    shutil.copy(src_valid_dat_path, os.path.join(notebook_source_dir_sc, dest_valid_cfg_name[:-4] + ".dat"))
else:
    print(f"ОШИБКА: signals_for_checker.cfg/dat не найдены в {os.path.join(comtrade_samples_path, 'specific_signals_1')}")

# 2. Файл, который НЕ пройдет стандартную проверку (например, neutral_current.cfg)
src_invalid_cfg_path = os.path.join(comtrade_samples_path, "specific_signals_1", "neutral_current.cfg")
src_invalid_dat_path = os.path.join(comtrade_samples_path, "specific_signals_1", "neutral_current.dat")
dest_invalid_cfg_name = "osc_sc_invalid_default_from_testdata.cfg"
if os.path.exists(src_invalid_cfg_path) and os.path.exists(src_invalid_dat_path):
    shutil.copy(src_invalid_cfg_path, os.path.join(notebook_source_dir_sc, dest_invalid_cfg_name))
    shutil.copy(src_invalid_dat_path, os.path.join(notebook_source_dir_sc, dest_invalid_cfg_name[:-4] + ".dat"))
else:
    print(f"ОШИБКА: neutral_current.cfg/dat не найдены (для теста 'invalid_default').")

# 3. Файл для пользовательской проверки (например, U и I на Bus-2)
custom_target_orig_path = os.path.join(comtrade_samples_path, "for_signal_checker_custom")
src_custom_target_cfg_path = os.path.join(custom_target_orig_path, "custom_target.cfg")
src_custom_target_dat_path = os.path.join(custom_target_orig_path, "custom_target.dat")
dest_custom_cfg_name = "osc_sc_custom_target_from_testdata.cfg"

if os.path.exists(src_custom_target_cfg_path):
    shutil.copy(src_custom_target_cfg_path, os.path.join(notebook_source_dir_sc, dest_custom_cfg_name))
    if os.path.exists(src_custom_target_dat_path):
         shutil.copy(src_custom_target_dat_path, os.path.join(notebook_source_dir_sc, dest_custom_cfg_name[:-4] + ".dat"))
    print(f"Скопирован файл для пользовательской проверки: {dest_custom_cfg_name}")
else:
    print(f"ОШИБКА: Файл для пользовательской проверки (custom_target.cfg) не найден в {custom_target_orig_path}. Создана заглушка.")
    dummy_custom_content = ["StationFail,DevFail,2013", "2,1A,1D", "1,U | Bus-1 | phase: A,,,,1,0,0,0,0,1,1,P", "2,RandomDigital,,,0", "50.0","1","1000,100","01/01/2023,00:00:00.000","01/01/2023,00:00:01.000","ASCII","1.0"]
    with open(os.path.join(notebook_source_dir_sc, dest_custom_cfg_name), "w", encoding="utf-8") as f:
        for line in dummy_custom_content: f.write(line + "\n")
    with open(os.path.join(notebook_source_dir_sc, dest_custom_cfg_name[:-4] + ".dat"), "w", encoding="utf-8") as f:
        f.write("1,0,0,0\n")

print(f"Демонстрационная директория для SignalChecker: {notebook_source_dir_sc}")
print("Содержимое:")
for item in os.listdir(notebook_source_dir_sc): print(f"- {item}")

## 1. Использование `SignalChecker` со стандартной логикой проверки
Стандартная логика (`_default_signal_check_logic`) проверяет наличие определенного набора напряжений, токов и сигналов PDR.

In [ ]:
checker_default = SignalChecker() 
default_report_path = os.path.join(notebook_output_dir_sc, "default_signal_check_report.csv")

print("\nПроверка сигналов со стандартной логикой:")
checker_default.check_signals_in_directory(
    source_dir=notebook_source_dir_sc,
    output_csv_path=default_report_path,
    verbose_logging_method=True, 
    show_progress_method=True
)

if os.path.exists(default_report_path):
    print(f"\nСодержимое отчета ({default_report_path}):")
    try: 
        from IPython.display import display
        display(pd.read_csv(default_report_path))
    except ImportError:
        print(pd.read_csv(default_report_path))


## 2. Использование `SignalChecker` с пользовательской логикой проверки
Можно передать свою функцию для проверки наличия сигналов.

In [ ]:
def custom_check_bus2_U_and_I(parsed_signals: list[dict]) -> bool:
    has_U_bus2 = False
    has_I_bus2 = False
    for signal in parsed_signals:
        loc_type = signal.get('location_type')
        sec_num = signal.get('section_number')
        sig_type = signal.get('signal_type')

        if loc_type == 'Bus' and sec_num == '2':
            if sig_type == 'U':
                has_U_bus2 = True
            elif sig_type == 'I':
                has_I_bus2 = True
        if has_U_bus2 and has_I_bus2:
            return True
    return False

custom_report_path_dir = os.path.join(notebook_output_dir_sc, "custom_signal_check_report.csv")

print("\nПроверка сигналов с пользовательской логикой (U и I на Bus-2) для всей директории:")
checker_custom_run = SignalChecker() # Можно и checker_default, если не хотим хранить custom_logic в нем
checker_custom_run.check_signals_in_directory(
    source_dir=notebook_source_dir_sc,
    output_csv_path=custom_report_path_dir,
    custom_signal_check_logic=custom_check_bus2_U_and_I, 
    verbose_logging_method=True,
    show_progress_method=True
)
if os.path.exists(custom_report_path_dir):
    print(f"\nСодержимое пользовательского отчета ({custom_report_path_dir}):")
    try: 
        from IPython.display import display
        display(pd.read_csv(custom_report_path))
    except ImportError:
        print(pd.read_csv(custom_report_path))

checker_for_file_check = SignalChecker(custom_signal_check_logic=custom_check_bus2_U_and_I)
custom_target_file_path = os.path.join(notebook_source_dir_sc, "osc_sc_custom_target_from_testdata.cfg")
valid_file_for_custom_check_path = os.path.join(notebook_source_dir_sc, "osc_sc_valid_from_testdata.cfg")

print("\nПроверка отдельных файлов с пользовательской логикой:")
if os.path.exists(custom_target_file_path):
    status_custom_target = checker_for_file_check.check_file_signals(custom_target_file_path, verbose_logging_method=True)
    print(f"Статус для {os.path.basename(custom_target_file_path)} (custom logic): {status_custom_target}")
else:
    print(f"Файл {custom_target_file_path} не найден для индивидуальной проверки.")

if os.path.exists(valid_file_for_custom_check_path):
    status_default_target_custom_check = checker_for_file_check.check_file_signals(valid_file_for_custom_check_path, verbose_logging_method=True)
    print(f"Статус для {os.path.basename(valid_file_for_custom_check_path)} (custom logic): {status_default_target_custom_check}")
else:
    print(f"Файл {valid_file_for_custom_check_path} не найден для индивидуальной проверки.")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(notebook_base_temp_dir_sc):
        shutil.rmtree(notebook_base_temp_dir_sc)
        print(f"\nВременная директория {notebook_base_temp_dir_sc} удалена.")
    else:
        print(f"\nВременная директория {notebook_base_temp_dir_sc} не найдена для удаления.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {notebook_base_temp_dir_sc}: {e}")